# Gn_num_test

지상 단말 개수에 따른 학습 시간(it/s), SE 변화 측정

In [1]:
from time import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import trange
import wandb

from datasets import CubeObstacle, CylinderObstacle, BlockageDataset
from utils.config import Hyperparameters as hp
from datasets import TrainDataset
from model import Net
from utils.tools import calc_loss

In [2]:
# Create datasets

batch_size = 2**10
random_seed = 42

obstacle_ls = [
    CubeObstacle(-30, 25, 35, 60, 20, 0.1),
    CubeObstacle(-30, -25, 45, 10, 35, 0.1),
    CubeObstacle(-30, -60, 35, 60, 20, 0.1),
    CubeObstacle(50, -20, 35, 25, 25, 0.1),
    CylinderObstacle(10, -5,  70, 15, 0.1),
]

obst_points = []
for obstacle in obstacle_ls:
    obst_points.append(torch.tensor(obstacle.points, dtype=torch.float32))

obst_points = torch.cat([op for op in obst_points], dim=1).mT.to(hp.device)

In [3]:
gn_datasets = []
for i in range(2,7):
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    gn_datasets.append(BlockageDataset(110000, obstacle_ls, i, dtype=torch.float32).to(hp.device))

100%|██████████| 110000/110000 [00:01<00:00, 70442.29it/s]


In [4]:
for dataset in gn_datasets:
    print(dataset.gnd_nodes.shape)

torch.Size([110000, 2, 3])
torch.Size([110000, 3, 3])
torch.Size([110000, 4, 3])
torch.Size([110000, 5, 3])
torch.Size([110000, 6, 3])


In [5]:
# data preprocessing
test_sets = {}

for idx, dataset in enumerate(gn_datasets):
    data = dataset.gnd_nodes.cpu().numpy()
    data = np.delete(data, 2, axis=2)
    data = data.reshape(-1, data.shape[1] * data.shape[2])

    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)
    test_data = data_scaled[100000:]
    train_data, val_data = train_test_split(data_scaled[:100000], test_size=0.2, random_state=random_seed)

    model = Net(data.shape[1], 1024, 4, output_N=2).to(hp.device)
    train_dataset = TrainDataset(train_data, dtype=torch.float32).to(hp.device)
    val_dataset = TrainDataset(val_data, dtype=torch.float32).to(hp.device)
    test_dataset = TrainDataset(test_data, dtype=torch.float32).to(hp.device)

    test_sets[idx] = {
        "gn_num": dataset.gnd_nodes.shape[1],
        "train_dataset": train_dataset,
        "val_dataset": val_dataset,
        "test_dataset": test_dataset,
        "scaler": scaler,
        "model": model,
        "result": {"train_loss": [], "val_loss": []}
    }


In [6]:
# Training

for idx, dataset in test_sets.items():
    wandb.init(project="DL-based UAV Positioning", name=f"gnd_test: {dataset['gn_num']}", config={
            "batch_size": batch_size,
            "epochs": 1000,
            "random_seed": random_seed,
            "learning_rates": 1e-4,
            "gn_num": dataset['gn_num']
        })

    train_dataloader = DataLoader(dataset["train_dataset"], batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(dataset["val_dataset"], batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(dataset["test_dataset"], batch_size=batch_size, shuffle=False)

    scaler_x = dataset["scaler"]

    # 모델 및 옵티마이저 초기화
    model = dataset["model"]
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    for epoch in trange(1000, desc=f"Training with gn_num={dataset['gn_num']}"):
        train_loss = 0.0
        model.train()
        start_time = time()
        for x in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(x)

            # x_reshaped 생성
            x_reshaped = torch.tensor(scaler_x.inverse_transform(x.cpu()), device=hp.device,
                                          dtype=torch.float32).view(-1, dataset["gn_num"], 2)
            x_reshaped = torch.cat(
                (x_reshaped, torch.zeros((x_reshaped.shape[0], x_reshaped.shape[1], 1), device=hp.device)), dim=-1)

            # y_pred 수정 및 손실 계산
            y_pred = torch.hstack((y_pred, torch.ones(y_pred.shape[0], 1, device=hp.device) * 0.7)) * 100
            loss = calc_loss(y_pred, x_reshaped, obst_points)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        end_time = time()


        # 검증 손실 계산
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for x in val_dataloader:
                y_pred = model(x)
                x_reshaped = torch.tensor(scaler_x.inverse_transform(x.cpu()), device=hp.device,
                                          dtype=torch.float32).view(-1, dataset["gn_num"], 2)
                x_reshaped = torch.cat(
                    (x_reshaped, torch.zeros((x_reshaped.shape[0], x_reshaped.shape[1], 1), device=hp.device)),
                    dim=-1)
                y_pred = torch.hstack((y_pred, torch.ones(y_pred.shape[0], 1, device=hp.device) * 0.7)) * 100
                val_loss += calc_loss(y_pred, x_reshaped, obst_points).item()

        # 에폭별 평균 손실 기록
        train_loss /= len(train_dataloader)
        val_loss /= len(val_dataloader)
        dataset["result"]["train_loss"].append(train_loss)
        dataset["result"]["val_loss"].append(val_loss)

        # wandb에 손실 로깅
        wandb.log({
            f"train_loss": train_loss,
            f"val_loss": val_loss,
            "epoch": epoch + 1,
            "time": end_time - start_time
        })

    # Test

    test_se = 0.0

    for x in test_dataloader:
        y_pred = model(x)
        x_reshaped = torch.tensor(scaler_x.inverse_transform(x.cpu()), device=hp.device,
                                  dtype=torch.float32).view(-1, dataset["gn_num"], 2)
        x_reshaped = torch.cat(
            (x_reshaped, torch.zeros((x_reshaped.shape[0], x_reshaped.shape[1], 1), device=hp.device)), dim=-1)
        y_pred = torch.hstack((y_pred, torch.ones(y_pred.shape[0], 1, device=hp.device) * 0.7)) * 100
        test_se -= calc_loss(y_pred, x_reshaped, obst_points).item()

    test_se /= len(test_dataloader)
    wandb.log({"test SE avg": test_se})
    wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: marvic1130. Use `wandb login --relogin` to force relogin


Training with gn_num=2: 100%|██████████| 1000/1000 [08:20<00:00,  2.00it/s]


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
test SE avg,▁
time,▆▁█▄▂▂▂▂▁▁▃▃▁▂▂▃▃▂▃▁▄▃▃▂▂▂▃▂▂▁▁▂▃▃▃▂▁▂▁▂
train_loss,▇▆▆▆█▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▂▁
val_loss,▇█▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▂▁▁▁▁
epoch,1000
test SE avg,12.39027
time,0.43371
train_loss,-12.36136
val_loss,-12.37478


Training with gn_num=3: 100%|██████████| 1000/1000 [10:49<00:00,  1.54it/s]


epoch,▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇█████
test SE avg,▁
time,▄▄▆▂▅▃▆▇▅▅▇▁▅▄▅▇▅▇▃▂▅▆▇▅▆▃▄█▃▃▃█▆█▆▅▅█▁▂
train_loss,█▄▅▄▅▃▃▃▄▃▃▃▃▃▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
val_loss,█▄▃▃▃▃▂▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁
epoch,1000
test SE avg,12.16282
time,0.58567
train_loss,-12.15517
val_loss,-12.17135


...

epoch,▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇██
test SE avg,▁
time,▂▁█▂▁▂▂▂▁▃▂▃▃▂▁▂▁▂▂▁▁▁▂▂▃▁▃▃▃▃▃▃▂▇▁▂▂▂▁▁
train_loss,█▆▆▅▅▅▄▄▄▄▄▅▄▄▄▃▃▂▃▂▃▃▃▃▂▂▂▃▂▂▁▂▃▂▂▁▂▂▁▁
val_loss,██▅▆▅▄▃▄▃▄▃▄▄▃▃▃▂▃▃▃▂▃▂▃▄▂▂▂▃▂▂▂▂▁▁▂▁▁▁▂
epoch,1000
test SE avg,12.07526
time,0.75303
train_loss,-12.0524
val_loss,-12.05723


Training with gn_num=5: 100%|██████████| 1000/1000 [15:37<00:00,  1.07it/s]


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇███
test SE avg,▁
time,▂▂▂▃▂▂▂▃▁▁▁▂▁▁▂▁▁▃▃▂▁▃▁▁▃█▁▂▃▂▃▇▂▂▃▃▂▂▁▁
train_loss,█▅▄▃▃▂▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▃▃▃▃▄▃▃▃▃▂▃▃▃▁▂▂▃▂▂▂▂▂▂▁▁▁▁▁▂▂▁▁▁▁▁▁▁▂
epoch,1000
test SE avg,12.0009
time,0.83638
train_loss,-12.00913
val_loss,-12.00774


Training with gn_num=6: 100%|██████████| 1000/1000 [17:52<00:00,  1.07s/it]


epoch,▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇███
test SE avg,▁
time,▃█▂▃▂▂▃▂▂▂▃█▂▁▁▂▃▃▃▃▁▂▃▂▃▁▃▃▁▂▁▂▂▂▃▁▁▁▃▃
train_loss,█▆▇▆▆▄▅▆▄▄▄▄▄▄▄▃▄▃▃▂▄▃▃▃▂▂▂▂▂▁▃▂▂▂▁▁▂▁▁▁
val_loss,█▆▅▅▆▃▃▃▃▃▂▃▂▃▂▂▂▂▂▁▂▃▂▂▂▂▁▁▂▂▂▁▁▂▂▁▁▂▂▁
epoch,1000
test SE avg,11.96867
time,0.96588
train_loss,-11.97645
val_loss,-11.97755
